In [15]:
# I: Standard library imports
import os
import warnings

# II: Third-party imports

# II.A: General utilities
import holidays
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# II.B: Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# III: Machine learning

# III.A: Scikit-learn
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, BaseCrossValidator, ParameterGrid
from sklearn.preprocessing import StandardScaler

# III.B: TensorFlow
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv1D, Dense, Dropout, Flatten, Input, Lambda, LSTM, MaxPooling1D
)
from tensorflow.keras.models import Model

# III.C: XGBoost
from xgboost import XGBRegressor

# III.D: Time-series specific models
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# IV: Local imports
from pauls_data_loaders import loader_functions as lf

# V: Global config options

# Silence TensorFlow messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Pandas display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Suppress warnings
warnings.filterwarnings("ignore")


# Helper Functions

## Feature Engineering

In [26]:
def add_lag_features(df, column, lags):
    for lag in lags:
        df[f"{column}_lag_{lag}"] = df[column].shift(lag)
    return df

def add_rolling_avg_features(df, column, windows):
    for window in windows:
        df[f"{column}_rolling_avg_{window}"] = df[column].rolling(window).mean()
    df.fillna(0, inplace=True)
    return df

def add_time_features(df):
    """
    Adds time-based features to the dataframe based on the index (assumed to be datetime).
    Parameters:
        df (pd.DataFrame): The input dataframe.
    Returns:
        pd.DataFrame: The dataframe with time-based features added.
    """
    # Ensure the index is datetime
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        raise ValueError("Index must be of datetime type")
    
    # Extracting features from datetime index
    df['hour'] = df.index.hour  # Hour of the day
    df['day_of_week'] = df.index.dayofweek  # Day of the week (0 = Monday, 6 = Sunday)
    df['month'] = df.index.month  # Month (1 = January, 12 = December)
    df['quarter'] = df.index.quarter  # Quarter of the year (1-4)
    df['year'] = df.index.year  # Year

    return df

In [4]:
def add_engineered_features(df, config, standardize_columns=None):
    """
    Adds engineered features, holiday flags, and standardizes specified columns.
    Assumes that datetime is always the index column.
    Parameters:
        df (pd.DataFrame): The input dataframe.
        config (dict): Configuration dictionary with feature parameters.
        standardize_columns (list): List of columns to standardize.
    Returns:
        pd.DataFrame: The dataframe with standardized columns and engineered features.
    """
    # Standardize specified columns
    if standardize_columns:
        scaler = StandardScaler()
        df[standardize_columns] = scaler.fit_transform(df[standardize_columns])

    # Add lag features
    if "lag" in config:
        for col, lags in config["lag"].items():
            df = add_lag_features(df, col, lags)

    # Add rolling averages
    if "rolling_avg" in config:
        for col, windows in config["rolling_avg"].items():
            df = add_rolling_avg_features(df, col, windows)

    # Add time features if the flag is True
    if config.get("add_time_features", False):
        df = add_time_features(df)

    return df

## Data Manager

In [5]:
class dataManager:
    def __init__(self, config={"lag": [1, 2, 3], "rolling_avg": [3, 6, 12], "add_time_features": True}):
        plain_data = lf.load_all_data()
        # List of column names
        self.stations = [f"station_{i}" for i in range(1, 12)]
        self.zones = [f"zone_{i}" for i in range(1, 21)]

        # Configuration for engineered features
        full_config = {
            "lag": {col: config["lag"] for col in self.stations},  # Lags for all station columns
            "rolling_avg": {col: config["rolling_avg"] for col in self.stations},  # Rolling averages
            "add_time_features": config["add_time_features"],  # Turn time-based features on/off
        }

        self.data = add_engineered_features(plain_data, full_config, standardize_columns=self.stations)
        self.engineereds = self.data.columns.difference(self.stations + self.zones).tolist()
    
    def out_data(self, zone=None, plain=False):
        return_cols = self.stations
        if zone == None:
            return_cols = self.zones + return_cols
        elif isinstance(zone, str):
            return_cols = zone + return_cols
        elif isinstance(zone, int):
            return_cols = [f'zone_{zone}'] + return_cols
        else:
            raise TypeError
        if not plain:
            return_cols += self.engineereds
        return self.data[return_cols]

## Preprocessers

In [59]:
def create_sequences(data, sequence_length):  # For LSTM CNN
    # Get the total number of sequences
    num_sequences = len(data) - sequence_length
    
    # Pre-allocate arrays for X and y
    X = np.empty((num_sequences, sequence_length, data.shape[1] - 1))  # Exclude response column
    y = np.empty(num_sequences)
    
    # Fill in the X and y arrays
    with tqdm(total=num_sequences, desc=f'Slicing Sequences of Length {sequence_length} 💅🔪', colour='purple') as pbar:
        for i in range(num_sequences):
            X[i] = data.iloc[i:i + sequence_length, 1:].values  # Predictors
            y[i] = data.iloc[i + sequence_length, 0]  # Response
            pbar.update(1)
    
    return X, y

# Model Functions

In [62]:
def lstm_cnn(sequence_length, feature_dim):
    # define shape of input
    input_shape = (sequence_length, feature_dim)
    
    # construct input layer
    inputs = Input(shape=input_shape)

    # construct CNN layers
    cnn = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    cnn = MaxPooling1D(pool_size=2)(cnn)
    cnn = Flatten()(cnn)

    # Define a Lambda layer to reshape the CNN output for LSTM
    lstm_input = Lambda(lambda x: tf.expand_dims(x, axis=1))(cnn)
    
    # construct LSTM layers
    lstm = LSTM(64, return_sequences=False)(lstm_input)

    # Fully connected layers
    dense = Dense(128, activation='relu')(lstm)
    dense = Dropout(0.5)(dense)
    outputs = Dense(1, activation='sigmoid')(dense)  # For binary classification

    # Build and compile the model
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Return
    return model

# Cross Validation

In [6]:
# Custom expanding window splitter
class ExpandingWindowSplit(BaseCrossValidator):
    def __init__(self, n_splits=5):
        self.n_splits = n_splits

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

    def split(self, X, y=None, groups=None):
        n_samples = X.shape[0]
        fold_size = n_samples // (self.n_splits + 1)
        for i in range(1, self.n_splits + 1):
            train_end = i * fold_size
            test_start = train_end
            test_end = test_start + fold_size

            train_indices = list(range(0, train_end))
            test_indices = list(range(test_start, test_end))

            yield train_indices, test_indices

    def _iter_test_indices(self, X=None, y=None, groups=None):
        n_samples = X.shape[0]
        fold_size = n_samples // (self.n_splits + 1)
        for i in range(1, self.n_splits + 1):
            test_start = i * fold_size
            test_end = test_start + fold_size
            yield range(test_start, test_end)

In [107]:
# Prepare data
d = dataManager()
data = d.out_data(1)

# Prepare the model and parameter grid
model = XGBRegressor(objective="reg:squarederror")
param_grid = {
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "max_depth": [3, 5, 7, 9],
    "n_estimators": [50, 100, 150, 200]
}

# Define GridSearchCV with the custom cross-validator
cv = ExpandingWindowSplit(n_splits=5)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring="neg_root_mean_squared_error", verbose=2)

In [ ]:
# Run GridSearch
X = data.drop(columns=["zone_1"])
y = data["zone_1"].astype(float)
grid_search.fit(X, y)

# Best parameters
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE:", -grid_search.best_score_)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   0.2s
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   0.2s
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   0.3s
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   0.3s
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   0.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.3s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.3s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.5s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.5s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=150; total time=   0.3s
[CV] END ..learning_rate=0.01, max_depth=3, n_e

In [109]:
cv_results = pd.DataFrame(grid_search.cv_results_)

cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.258183,0.071776,0.027140,0.001077,0.01,3,50,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",-3931.778607,-4358.268534,-4248.822299,-5301.988923,-4285.224821,-4425.216637,462.040139,64
1,0.375293,0.096825,0.026897,0.000222,0.01,3,100,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",-3194.403227,-3538.405771,-3424.828146,-4265.655516,-3541.984799,-3593.055492,359.224072,60
2,0.476996,0.129552,0.030612,0.003065,0.01,3,150,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",-2763.687870,-3047.553706,-2952.439847,-3670.528305,-3113.790754,-3109.600097,304.257254,58
3,0.670661,0.142041,0.036325,0.002248,0.01,3,200,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",-2524.113289,-2747.701128,-2647.810651,-3311.599354,-2840.464362,-2814.337757,269.945792,55
4,0.427288,0.071033,0.032305,0.001886,0.01,5,50,"{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",-3703.243726,-4037.170863,-3966.934482,-5078.502568,-4039.777252,-4165.125778,473.065893,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2.188185,0.299092,0.045681,0.003455,0.20,7,200,"{'learning_rate': 0.2, 'max_depth': 7, 'n_esti...",-1893.050292,-1652.502320,-1630.379632,-1923.421571,-1627.270413,-1745.324846,133.646193,13
60,1.459452,0.211943,0.038731,0.002745,0.20,9,50,"{'learning_rate': 0.2, 'max_depth': 9, 'n_esti...",-1885.887953,-1649.770790,-1636.075941,-2014.394793,-1560.101347,-1749.246165,171.728498,19
61,2.897131,0.700630,0.044604,0.006786,0.20,9,100,"{'learning_rate': 0.2, 'max_depth': 9, 'n_esti...",-1888.718303,-1651.316388,-1636.657816,-2026.043121,-1541.532140,-1748.853554,179.710743,18
62,3.424773,0.388394,0.049104,0.006406,0.20,9,150,"{'learning_rate': 0.2, 'max_depth': 9, 'n_esti...",-1886.844899,-1652.364787,-1638.376670,-2036.857868,-1535.129304,-1749.914706,183.943331,20


In [ ]:
# Prepare data
d = dataManager()
data = d.out_data(1)

# Define the model
gbr = GradientBoostingRegressor()

# Define the parameter grid
param_grid = {
    "learning_rate": [0.05, 0.1, 0.2],
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "subsample": [0.8, 1.0]
}

# Define scoring function (RMSE)
scorer = make_scorer(mean_squared_error, squared=False)

# Use your ExpandingWindowSplit or any cross-validator
cv = ExpandingWindowSplit(n_splits=5)

# Split Data
X = data.drop(columns=["zone_1"])
y = data["zone_1"].astype(float)

# GridSearchCV setup
grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=cv, scoring=scorer, verbose=2)
grid_search.fit(X, y)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE:", grid_search.best_score_)

In [14]:
# Prepare data
d = dataManager()
data = d.out_data(1)

# Split Data
X = data.drop(columns=["zone_1"])
y = data["zone_1"].astype(float)

cv = ExpandingWindowSplit(n_splits=5)

# Prepare the parameter grid
param_grid = {
    "min_child_weight": [1, 3, 5],           # Minimum sum of instance weights (regularization)
    "subsample": [0.7, 0.8, 1.0],            # Fraction of samples for training each tree
    "colsample_bytree": [0.7, 0.8, 1.0],     # Fraction of features for each tree
    "gamma": [0, 0.1, 0.2],                  # Minimum loss reduction to make a split
    "reg_alpha": [0, 0.01, 0.1],             # L1 regularization (sparsity control)
    "reg_lambda": [1, 2, 5],                 # L2 regularization (ridge-style control)
}
grid = ParameterGrid(param_grid)

# Initialize tracking
best_params = None
best_score = np.inf  # RMSE should be minimized

# Custom progress bar
progress_bar = tqdm(grid, desc="Grid Search Progress", colour='pink', leave=True)

# Manual grid search loop
for params in progress_bar:
    # Update the progress bar with current params
    progress_bar.set_postfix(n_estimators=50, learning_rate=0.2, max_depth=3, params=params)

    # Configure the model with current parameters
    model = XGBRegressor(**params)

    # Cross-validation (using your custom CV splitter)
    scores = []
    for train_idx, test_idx in cv.split(X, y):
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        preds = model.predict(X.iloc[test_idx])
        scores.append(mean_squared_error(y.iloc[test_idx], preds, squared=False))
    
    # Average RMSE across folds
    avg_score = np.mean(scores)
    
    # Update best score
    if avg_score < best_score:
        best_score = avg_score
        best_params = params

# Display results
print("Best Parameters:", best_params)
print("Best RMSE:", best_score)

Grid Search Progress:   0%|          | 0/729 [00:00<?, ?it/s]

Best Parameters: {'colsample_bytree': 0.8, 'gamma': 0, 'min_child_weight': 3, 'reg_alpha': 0.1, 'reg_lambda': 2, 'subsample': 0.8}
Best RMSE: 1703.6925160391704


In [ ]:
# Prepare data
d = dataManager()
data = d.out_data(1)

# Define the model
gbr = XGBRegressor()

# Define the parameter grid
param_grid = {
    "n_estimators": [100, 200],
    "learning_rate": [0.05, 0.1],
    "max_depth": [5, 7],
    "min_child_weight": [1, 3],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 0.1],
    "reg_alpha": [0, 0.1],
    "reg_lambda": [1, 2],
}

# Define scoring function (RMSE)
scorer = make_scorer(mean_absolute_error)

# Use your ExpandingWindowSplit or any cross-validator
cv = ExpandingWindowSplit(n_splits=5)

# Split Data
X = data.drop(columns=["zone_1"])
y = data["zone_1"].astype(float)

# GridSearchCV setup
grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=cv, scoring=scorer, verbose=5)
grid_search.fit(X, y)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE:", grid_search.best_score_)

Fitting 5 folds for each of 512 candidates, totalling 2560 fits
[CV 1/5] END colsample_bytree=0.8, gamma=0, learning_rate=0.05, max_depth=5, min_child_weight=1, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.8;, score=1406.440 total time=   0.7s
[CV 2/5] END colsample_bytree=0.8, gamma=0, learning_rate=0.05, max_depth=5, min_child_weight=1, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.8;, score=1305.531 total time=   0.6s
[CV 3/5] END colsample_bytree=0.8, gamma=0, learning_rate=0.05, max_depth=5, min_child_weight=1, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.8;, score=1331.836 total time=   0.6s
[CV 4/5] END colsample_bytree=0.8, gamma=0, learning_rate=0.05, max_depth=5, min_child_weight=1, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.8;, score=1520.760 total time=   0.7s
[CV 5/5] END colsample_bytree=0.8, gamma=0, learning_rate=0.05, max_depth=5, min_child_weight=1, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.8;, score=1335

In [45]:
# Pull best model from GridSearch

with tqdm(total=20, desc=f'Predicting for all zones', colour='pink') as pbar:
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    stations = [f"station_{i}" for i in range(1, 12)]
    zones = [f"zone_{i}" for i in range(1, 21)]

    config = {
                "lag": {col: [1, 2, 3] for col in stations},  # Lags for all station columns
                "rolling_avg": {col: [3, 6, 12] for col in stations},  # Rolling averages
                "add_time_features": True,  # Turn time-based features on/off
            }

    pbar.set_description('engineering features')
    d_with_nans = add_engineered_features(lf.load_all_data(dropna=False), config, standardize_columns=stations)
    pbar.update(0.25)

    pbar.set_description('slicing data')
    backcast_data = d_with_nans['2005-03-06':'2005-03-13'] + d_with_nans['2005-06-20':'2005-06-27'] + d_with_nans['2005-09-10':'2005-09-17'] + d_with_nans['2005-09-10':'2005-09-17'] + d_with_nans['2005-12-25':'2006-01-01'] + d_with_nans['2006-02-13':'2006-02-20'] + d_with_nans['2006-05-25':'2006-06-01'] + d_with_nans['2006-08-02':'2006-08-09'] + d_with_nans['2006-11-22':'2006-11-29']

    backcast_data = backcast_data[data.columns]
    backcast_data.drop(columns=['zone_1'], inplace=True)
    pbar.update(0.25)
    
    pbar.set_description('Predicting zone_1')
    backcast_loads = {}
    backcast_loads['zone_1'] = best_model.predict(backcast_data)

    backcast_scores = {}
    backcast_scores['zone_1'] = grid_search.best_score_

    print(f'zone_1: MAE={backcast_scores['zone_1'] } vals={backcast_loads['zone_1']}')
    pbar.update(0.5)

    # more for zones 2-20
    for z in range (2,21):
        zone = f'zone_{z}'
        z_cols = [col for col in zones if col != zone]
        zone_data = d_with_nans.drop(columns=z_cols, axis=1).copy()
        zone_data = zone_data[sorted(zone_data.columns)]

        # Split Data
        X = zone_data.drop(columns=[zone])
        y = zone_data[zone].astype(float)

        # Configure the model with current parameters
        model = XGBRegressor(**best_params)
        cv = ExpandingWindowSplit(n_splits=5)

        # Cross-validation (using your custom CV splitter)
        pbar.set_description(f'Training {zone}')
        scores = []
        for train_idx, test_idx in cv.split(X, y):
            model.fit(X.iloc[train_idx], y.iloc[train_idx])
            preds = model.predict(X.iloc[test_idx])
            scores.append(mean_absolute_error(y.iloc[test_idx], preds))
            pbar.update(0.15)
        
        # Average RMSE across folds
        backcast_scores[zone] = np.mean(scores)

        backcast_data = backcast_data[sorted(backcast_data.columns)]

        pbar.set_description(f'Predicting {zone}')
        backcast_loads[zone] = model.predict(backcast_data)

        print(f'{zone}: MAE={backcast_scores[zone] } vals={backcast_loads[zone]}')
        pbar.update(0.25)

Predicting for all zones:   0%|          | 0/20 [00:00<?, ?it/s]

zone_1: MAE=1395.875516646966 vals=[36868.79 36868.79 36868.79 ... 36868.79 36868.79 36868.79]
zone_2: MAE=24049.78147093018 vals=[187839.9 187839.9 187839.9 ... 187839.9 187839.9 187839.9]
zone_3: MAE=25920.84738164752 vals=[204693.48 204693.48 204693.48 ... 204693.48 204693.48 204693.48]
zone_4: MAE=75.79200039478751 vals=[313.69916 313.69916 313.69916 ... 313.69916 313.69916 313.69916]
zone_5: MAE=1194.6654461108978 vals=[8283.114 8283.114 8283.114 ... 8283.114 8283.114 8283.114]
zone_6: MAE=24793.15622563052 vals=[178316.38 178316.38 178316.38 ... 178316.38 178316.38 178316.38]
zone_7: MAE=25920.84738164752 vals=[204693.48 204693.48 204693.48 ... 204693.48 204693.48 204693.48]
zone_8: MAE=576.5472960134866 vals=[4902.7163 4902.7163 4902.7163 ... 4902.7163 4902.7163 4902.7163]
zone_9: MAE=17017.067723182867 vals=[37407.996 37407.996 37407.996 ... 37407.996 37407.996 37407.996]
zone_10: MAE=10071.14764866797 vals=[38058.242 38058.242 38058.242 ... 38058.242 38058.242 38058.242]
zone_

# LSTM stuff

def lstm_cnn_cv(data, sequence_lengths, n_splits=5):
    results = {}

    num_models = len(sequence_lengths)
    with tqdm(total=num_models, desc=f'cv on sequence_lengths: {sequence_lengths}', colour='pink') as pbar1:
        for seq_len in sequence_lengths:
            n = len(data)
            fold_size = n // (n_splits + 1)
            zone = 'zone_1'

            opt_results = []

            with tqdm(total=n_splits, desc=f'Validating {seq_len}', colour='pink') as pbar:
                for i in range(1, n_splits + 1):
                    train_end = i * fold_size
                    test_start = train_end
                    test_end = test_start + fold_size

                    pbar.set_description(f'Validating [splitting data] {seq_len}')
                    train = data.iloc[:train_end]
                    test = data.iloc[test_start:test_end]

                    # Preprocess the data after splitting (train/test separately)
                    X_train, y_train = train.drop(columns=zone), train[zone]
                    X_test, y_test = test.drop(columns=zone), test[zone]

                    pbar.set_description(f'Validating [splitting training data] {seq_len}')
                    X_train, y_train = create_sequences(train, seq_len)  # Pass preprocessing params
                    pbar.set_description(f'Validating [splitting testing data] {seq_len}')
                    X_test, y_test = create_sequences(test, seq_len)

                    pbar.set_description(f'Validating [training model] {seq_len}')
                    model = lstm_cnn(seq_len, 82)  # Pass model params to the model function
                    model.fit(X_train, y_train, epochs=5, verbose=0)

                    pbar.set_description(f'Validating [evaluating model] {seq_len}')
                    predictions = model.predict(X_test, verbose=0)
                    rmse = np.sqrt(mean_squared_error(y_test, predictions))
                    opt_results.append(rmse)
                    pbar.update(1)
                results[seq_len] = np.mean(opt_results)
            pbar1.update(1)
        pbar.set_description('Finished!')
    return results

d = dataManager()

results = lstm_cnn_cv(d.out_data(1), [24])

results